In [90]:
#r "nuget: Microsoft.Spark,1.0.0"

In [91]:
using Microsoft.Spark.Sql;

In [92]:
 SparkSession sparkSession = SparkSession.Builder().AppName("Street Counter").GetOrCreate();

In [93]:
 DataFrame dfCsv =
                sparkSession
                    .Read()
                    .Option("delimiter", ";")
                    .Schema("WOJ string ,POW string ,GMI string ,RODZ_GMI string , " +
                            "SYM string , SYM_UL string , " +
                            "CECHA string , NAZWA_1 string ,NAZWA_2 string , " +
                            "STAN_NA string")
                    .Csv("streets.csv");

In [94]:
DataFrame dataIn = dfCsv
                .WithColumn("STREET", Functions.ConcatWs(" ", dfCsv["CECHA"], dfCsv["NAZWA_1"], dfCsv["NAZWA_2"]));

In [95]:
DataFrame dataGroup = dataIn
                .Select("STREET")
                .GroupBy("STREET")
                .Count()
                .WithColumnRenamed("count","COUNT");

In [96]:
DataFrame dataOut = dataGroup
                .OrderBy(dataGroup["COUNT"]
                    .Desc()
                );

In [ ]:
dataOut.Show()

In [98]:
#r "nuget:XPlot.Plotly,2.0.0"

In [99]:
var first = dataOut.Take(10);
var streets = first.Select(a=>a.GetAs<string>("STREET"));
var counts = first.Select(a=>a.GetAs<int>("COUNT"));

In [100]:
using XPlot.Plotly;

In [ ]:
display(Chart.Plot(new Graph.Bar(){x=streets , y = counts}));

In [ ]:
display(Chart.Plot(new Graph.Pie(){values=counts , labels = streets}));